In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 9章 整形
## 9-1 縦持ちデータから横持ちデータへの変換
### Q: 横持ち変換
#### Not Awesome

In [ ]:
%%bigquery
select
    hotel_id,
    -- (2) 宿泊人数ごとの予約数を集計
    count(case when people_num = 1 then 1 end) as people_num_1,
    count(case when people_num = 2 then 1 end) as people_num_2,
    count(case when people_num = 3 then 1 end) as people_num_3,
    count(case when people_num = 4 then 1 end) as people_num_4
from example.reservation
-- (1) ホテルごとにgroup by
group by hotel_id

#### Not Awesome

In [ ]:
%%bigquery
select *
from (
    -- （1）横持ちデータの行になるhotel_id、列になるpeople_numのみ抽出
    select
        hotel_id,
        people_num
    from example.reservation
)
-- （2）people_numを列、残りのhotel_idを行としてpivot集計
pivot (count(*) as people_num for people_num in (1, 2, 3, 4))

## 9-2 横持ちから縦持ちへの変換
### Q: 縦持ち変換
#### Not Awesome

In [ ]:
%%bigquery
-- （1）business列のデータ抽出
select
    month,
    "business" as hotel_type,
    business as sales
from example.hotel_sales

union all

-- （2）minshuku列のデータ抽出
select
    month,
    "minshuku" as hotel_type,
    minshuku as sales
from example.hotel_sales

union all

-- （3）resort列のデータ抽出
select
    month,
    "resort" as hotel_type,
    resort as sales
from example.hotel_sales

union all

-- （4）ryokan列のデータ抽出
select
    month,
    "ryokan" as hotel_type,
    ryokan as sales
from example.hotel_sales

#### Not Awesome

In [ ]:
%%bigquery
select *
from example.hotel_sales
unpivot (sales for hotel_type in (business, minshuku, resort, ryokan))